In [1]:
import pandas as pd

In [2]:
# I split the input data into two data sets
tags = pd.read_csv('testtags.csv')
file = pd.read_csv('Sample Input - Sample.csv.csv')

In [3]:
file.shape

(12751, 22)

In [4]:
file.columns

Index(['Date', 'Time', 'Milli Sec', 'L_R1SDW_SPDREF', 'L_R1SDD_SPDREF',
       'L_R1SDL_SPDREF', 'L_R1SDW_SPDFBK', 'L_R1SDD_SPDFBK', 'L_R1SDL_SPDFBK',
       'L_R1_ODG_AUX_B07', 'RM1_RUNDOWN_ID_HS', 'L_R1SD_WS_GAP',
       'L_R1SD_DS_GAP', 'L_R1SD_WS_SPDREF', 'L_R1SD_DS_SPDREF',
       'L_R1SDLS_SPDREF', 'L_R1SD_WS_SPDFBK', 'L_R1SD_DS_SPDFBK',
       'L_R1SDLS_SPDFBK', 'L_R1SD_WS_CUR', 'L_R1SD_DS_CUR', 'L_R1_CNT_PASS0'],
      dtype='object')

In [5]:
tags.columns

Index(['ItemId', 'ComputerId', 'ItemName', 'Comment', 'Unit', 'C/S/ID',
       'DataType', 'BitPos', 'Address', 'MAX', 'MIN', 'ScaleMax', 'ScaleMin',
       'Decimal'],
      dtype='object')

In [6]:
# First step towards transformation joining date and time columns to event_timestampcolumn

file['event_timestamp'] = pd.to_datetime(file.Date.astype(str)+' '+file.Time.astype(str))

In [7]:
# dropping unrequired columns
file.drop(['Time', 'Date', 'Milli Sec'], axis= 1, inplace= True)

In [8]:
# resampling and aggregating using the max function
resamp = file.resample('10S', on = 'event_timestamp').max().reset_index(drop= True)

In [9]:
resamp

,L_R1SDW_SPDREF,L_R1SDD_SPDREF,L_R1SDL_SPDREF,L_R1SDW_SPDFBK,L_R1SDD_SPDFBK,L_R1SDL_SPDFBK,L_R1_ODG_AUX_B07,RM1_RUNDOWN_ID_HS,L_R1SD_WS_GAP,L_R1SD_DS_GAP,L_R1SD_WS_SPDREF,L_R1SD_DS_SPDREF,L_R1SDLS_SPDREF,L_R1SD_WS_SPDFBK,L_R1SD_DS_SPDFBK,L_R1SDLS_SPDFBK,L_R1SD_WS_CUR,L_R1SD_DS_CUR,L_R1_CNT_PASS0,event_timestamp
0,0.000000,0.000000,0,0.0000,0.000000,0,1,225726050,194.680008,194.619995,0.000000,0.000000,0,0.0000,0.000000,0,0,0,1,2022-06-05 21:58:39
1,29.866667,29.866667,0,16.7824,16.812799,0,1,225726050,194.678009,194.612000,29.866667,29.866667,0,16.7824,16.812799,0,177,213,0,2022-06-05 21:58:49
2,0.000000,0.000000,0,0.1552,0.148800,0,1,225726050,167.284012,167.197998,0.000000,0.000000,0,0.1552,0.148800,0,57,54,0,2022-06-05 21:58:59
3,14.952726,14.952726,0,10.5504,10.556801,0,1,225726050,167.266006,167.185989,14.952726,14.952726,0,10.5504,10.556801,0,187,205,0,2022-06-05 21:59:09
4,0.000000,0.000000,0,0.0000,0.000000,0,1,225726050,154.500000,154.487991,0.000000,0.000000,0,0.0000,0.000000,0,46,46,0,2022-06-05 21:59:19
5,0.000000,0.000000,0,0.0000,0.000000,0,1,225726050,192.596008,192.447998,0.000000,0.000000,0,0.0000,0.000000,0,189,172,1,2022-06-05 21:59:29
6,0.000000,0.000000,0,0.0000,0.000000,0,1,225726050,194.502014,194.419998,0.000000,0.000000,0,0.0000,0.000000,0,51,52,1,2022-06-05 21:59:36


In [10]:
# I assume the value in the tag__value is the max item so the one tag for each row is the tag that has the maximum value.

tag = []
val = []

# basically i loop through the rows and get column name of row with maximum value whick i use as the tag value
for ind, row in resamp.drop('event_timestamp', axis= 1).iterrows():
    row_ind = row.index
    row_val = row.values
    inde = list(row_val).index(max(row_val))
    tag.append(row_ind[inde])
    val.append(max(row_val))

resamp['tag__name'] = tag
resamp['tag__value'] = val

In [11]:
resamp

,L_R1SDW_SPDREF,L_R1SDD_SPDREF,L_R1SDL_SPDREF,L_R1SDW_SPDFBK,L_R1SDD_SPDFBK,L_R1SDL_SPDFBK,L_R1_ODG_AUX_B07,RM1_RUNDOWN_ID_HS,L_R1SD_WS_GAP,L_R1SD_DS_GAP,...,L_R1SDLS_SPDREF,L_R1SD_WS_SPDFBK,L_R1SD_DS_SPDFBK,L_R1SDLS_SPDFBK,L_R1SD_WS_CUR,L_R1SD_DS_CUR,L_R1_CNT_PASS0,event_timestamp,tag__name,tag__value
0,0.000000,0.000000,0,0.0000,0.000000,0,1,225726050,194.680008,194.619995,...,0,0.0000,0.000000,0,0,0,1,2022-06-05 21:58:39,RM1_RUNDOWN_ID_HS,225726050.0
1,29.866667,29.866667,0,16.7824,16.812799,0,1,225726050,194.678009,194.612000,...,0,16.7824,16.812799,0,177,213,0,2022-06-05 21:58:49,RM1_RUNDOWN_ID_HS,225726050.0
2,0.000000,0.000000,0,0.1552,0.148800,0,1,225726050,167.284012,167.197998,...,0,0.1552,0.148800,0,57,54,0,2022-06-05 21:58:59,RM1_RUNDOWN_ID_HS,225726050.0
3,14.952726,14.952726,0,10.5504,10.556801,0,1,225726050,167.266006,167.185989,...,0,10.5504,10.556801,0,187,205,0,2022-06-05 21:59:09,RM1_RUNDOWN_ID_HS,225726050.0
4,0.000000,0.000000,0,0.0000,0.000000,0,1,225726050,154.500000,154.487991,...,0,0.0000,0.000000,0,46,46,0,2022-06-05 21:59:19,RM1_RUNDOWN_ID_HS,225726050.0
5,0.000000,0.000000,0,0.0000,0.000000,0,1,225726050,192.596008,192.447998,...,0,0.0000,0.000000,0,189,172,1,2022-06-05 21:59:29,RM1_RUNDOWN_ID_HS,225726050.0
6,0.000000,0.000000,0,0.0000,0.000000,0,1,225726050,194.502014,194.419998,...,0,0.0000,0.000000,0,51,52,1,2022-06-05 21:59:36,RM1_RUNDOWN_ID_HS,225726050.0


In [12]:
# to get the tag id and descrption, I'll do a merge on the resamp and tags dataframe

data = pd.merge(resamp, tags, how= 'left', left_on= 'tag__name', right_on= 'ItemName')

In [13]:
data

,L_R1SDW_SPDREF,L_R1SDD_SPDREF,L_R1SDL_SPDREF,L_R1SDW_SPDFBK,L_R1SDD_SPDFBK,L_R1SDL_SPDFBK,L_R1_ODG_AUX_B07,RM1_RUNDOWN_ID_HS,L_R1SD_WS_GAP,L_R1SD_DS_GAP,...,Unit,C/S/ID,DataType,BitPos,Address,MAX,MIN,ScaleMax,ScaleMin,Decimal
0,0.000000,0.000000,0,0.0000,0.000000,0,1,225726050,194.680008,194.619995,...,NaN,0,TEXT : Text Data,0,0x26db0,NaN,NaN,NaN,NaN,NaN
1,29.866667,29.866667,0,16.7824,16.812799,0,1,225726050,194.678009,194.612000,...,NaN,0,TEXT : Text Data,0,0x26db0,NaN,NaN,NaN,NaN,NaN
2,0.000000,0.000000,0,0.1552,0.148800,0,1,225726050,167.284012,167.197998,...,NaN,0,TEXT : Text Data,0,0x26db0,NaN,NaN,NaN,NaN,NaN
3,14.952726,14.952726,0,10.5504,10.556801,0,1,225726050,167.266006,167.185989,...,NaN,0,TEXT : Text Data,0,0x26db0,NaN,NaN,NaN,NaN,NaN
4,0.000000,0.000000,0,0.0000,0.000000,0,1,225726050,154.500000,154.487991,...,NaN,0,TEXT : Text Data,0,0x26db0,NaN,NaN,NaN,NaN,NaN
5,0.000000,0.000000,0,0.0000,0.000000,0,1,225726050,192.596008,192.447998,...,NaN,0,TEXT : Text Data,0,0x26db0,NaN,NaN,NaN,NaN,NaN
6,0.000000,0.000000,0,0.0000,0.000000,0,1,225726050,194.502014,194.419998,...,NaN,0,TEXT : Text Data,0,0x26db0,NaN,NaN,NaN,NaN,NaN


In [14]:
# selecting required columns

data = data[['event_timestamp', 'ItemId', 'tag__name', 'Comment', 'tag__value']]

In [15]:
# function to get the tag unit
# for some other tags where there are units we have them start with a parenthesis so i basically look for the first brace of the
# parenthesis, slice values from that parenthesis forward and remove the parenthesis which leaves the unit. If there's no 
# parenthesis we return nothing meaning there's no unit.

def tag_unit(val):
    for t,v in enumerate(val):
        if v == '(':
            unit = val[t:]
            unit = unit.replace('(' , '')
            unit = unit.replace(')', '')
            
            return unit
        else:
            return ''
    


In [16]:
data['tag__unit'] = data['Comment'].map(tag_unit)

<ipython-input-16-3f662f9a4771>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['tag__unit'] = data['Comment'].map(tag_unit)


In [17]:
data

,event_timestamp,ItemId,tag__name,Comment,tag__value,tag__unit
0,2022-06-05 21:58:39,83528,RM1_RUNDOWN_ID_HS,R! RUNDOWN ID,225726050.0,
1,2022-06-05 21:58:49,83528,RM1_RUNDOWN_ID_HS,R! RUNDOWN ID,225726050.0,
2,2022-06-05 21:58:59,83528,RM1_RUNDOWN_ID_HS,R! RUNDOWN ID,225726050.0,
3,2022-06-05 21:59:09,83528,RM1_RUNDOWN_ID_HS,R! RUNDOWN ID,225726050.0,
4,2022-06-05 21:59:19,83528,RM1_RUNDOWN_ID_HS,R! RUNDOWN ID,225726050.0,
5,2022-06-05 21:59:29,83528,RM1_RUNDOWN_ID_HS,R! RUNDOWN ID,225726050.0,
6,2022-06-05 21:59:36,83528,RM1_RUNDOWN_ID_HS,R! RUNDOWN ID,225726050.0,


In [18]:
data.rename({'ItemId': 'tag__id',  'Comment': 'tag__desc'}, inplace= True, axis= 1)

C:\Users\Chigozie\anaconda3\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [19]:
data

,event_timestamp,tag__id,tag__name,tag__desc,tag__value,tag__unit
0,2022-06-05 21:58:39,83528,RM1_RUNDOWN_ID_HS,R! RUNDOWN ID,225726050.0,
1,2022-06-05 21:58:49,83528,RM1_RUNDOWN_ID_HS,R! RUNDOWN ID,225726050.0,
2,2022-06-05 21:58:59,83528,RM1_RUNDOWN_ID_HS,R! RUNDOWN ID,225726050.0,
3,2022-06-05 21:59:09,83528,RM1_RUNDOWN_ID_HS,R! RUNDOWN ID,225726050.0,
4,2022-06-05 21:59:19,83528,RM1_RUNDOWN_ID_HS,R! RUNDOWN ID,225726050.0,
5,2022-06-05 21:59:29,83528,RM1_RUNDOWN_ID_HS,R! RUNDOWN ID,225726050.0,
6,2022-06-05 21:59:36,83528,RM1_RUNDOWN_ID_HS,R! RUNDOWN ID,225726050.0,


In [20]:
data.to_csv('Sample Output.csv')